In [0]:
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
import pandas as pd
import math
import torch
from tqdm import tqdm
import torch.optim as optim
import matplotlib.pyplot as plt

In [0]:
# Build Dataset
class DiabetesDataset(Dataset):
  def __init__(self):
    self.data = pd.read_csv('diabetes.csv')

  def __getitem__(self, idx):
    return self.data.iloc[idx]
  
  def __len__(self):
    return len(self.data)


# Override the __getitem__ to return tensors

class TensorDiabetesDataset(DiabetesDataset):
  def __getitem__(self, idx):
    sample = super().__getitem__(idx)
    return {
        'X': torch.Tensor([
              sample.Pregnancies,
              sample.Glucose,
              sample.BloodPressure,
              sample.SkinThickness,
              sample.Insulin,
              sample.BMI,
              sample.Age,
              sample.DiabetesPedigreeFunction
        ]),
        'Y': sample.Outcome
    }

In [0]:
TensorDiabetesDataset()[1]

FileNotFoundError: ignored

In [0]:
# Build Validation and Training samples
dataset = TensorDiabetesDataset()
noOfSamples = len(dataset)
batch_size = 10
validationSplit = .2
shuffleSample = True
totalNumberOfValidationSamples = int(math.floor(noOfSamples * validationSplit))
indices = [ i for i in range(0, noOfSamples)]

if shuffleSample:
  np.random.shuffle(indices)

val_indices, train_indices = indices[:totalNumberOfValidationSamples], indices[totalNumberOfValidationSamples:]

# Create the Samples
val_sampler = SubsetRandomSampler(val_indices)
train_sampler = SubsetRandomSampler(train_indices)

# Create Datasets
traindataset = DataLoader(dataset, batch_size=batch_size, sampler=train_sampler)
valdataset = DataLoader(dataset, batch_size=batch_size, sampler=val_sampler)

FileNotFoundError: ignored

In [0]:
# Define Neural Network
class Net(nn.Module):
  def __init__(self):
    super().__init__()
    self.fc1 = nn.Linear(8, 10)
    self.fc2 = nn.Linear(10, 10)
    self.fc3 = nn.Linear(10, 10)
    self.fc4 = nn.Linear(10, 2)

  def forward(self, inputs):
    buffer = F.relu(self.fc1(inputs))
    buffer = F.relu(self.fc2(buffer))
    buffer = F.relu(self.fc3(buffer))
    
    buffer = self.fc4(buffer)
    return buffer

  def predict(self, x):
    pred = F.softmax(self.forward(x))
    answers = []

    for t in pred: 
      if t[0] > t[1]:
        answers.append(0)
      else:
        answers.append(1)

    return torch.Tensor(answers)


net = Net()

In [0]:
# Training Loop

# Define optimizer
optimizer = optim.Adam(net.parameters(), lr=0.001)

EPOCHS= 5
epoch_list = list(range(EPOCHS))
loss_list = []

for epoch in range(EPOCHS):
  for batch in traindataset:
    X, y = batch['X'], batch['Y']
    net.zero_grad()
    outputs = net.forward(X.view(-1,8))
    loss = F.mse_loss(outputs, y.view(-1, 1))
    loss.backward()
    optimizer.step()
  loss_list.append(loss)

plt.style.use("ggplot")
plt.xlabel('EPOCH')
plt.ylabel('LOSS')

plt.plot(epoch_list, loss_list)

In [0]:
correct = 0
total = 0
with torch.no_grad():
  for data in valdataset:
    X, y = data['X'], data['Y']
    output = net.predict(X)
    for idx, i in enumerate(output):
      if  i == y[idx]:
        correct +=1
      total +=1   

print("Accuracy: {}%".format(correct/total))

NameError: ignored

In [0]:
a = [1.0000, 85.0000, 66.0000, 29.0000,  0.0000, 26.6000, 31.0000,  0.3510]
b = torch.Tensor(a)
torch.argmax(net(b))

NameError: ignored